In [8]:
import pandas as pd
import numpy as np
from importlib import reload
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score, KFold, StratifiedKFold
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, mean_squared_error,r2_score
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.impute import SimpleImputer
import argparse
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
from imblearn.metrics import classification_report_imbalanced
from collections import Counter
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA, TruncatedSVD
import matplotlib.patches as mpatches
import pickle
import time
from sklearn.pipeline import  Pipeline, make_pipeline

from transformation import RemoveSkewnessKurtosis, Standardize, ImputeNumeric


from sklearn.utils.validation import column_or_1d

# Import Data
path = '/home/daisy/FDA_Dataset/inpatient_CVD_final_1.csv'
data = pd.read_csv(path).iloc[:,1:]
X = data.drop(columns = ['CVD_readmission', 'readmission within 300 days'])
y = column_or_1d(data[['readmission within 300 days']])


# Split Train and Test
transform_steps = [("ImputeNumeric", ImputeNumeric()),
                   ('RemoveSkewnessKurtosis', RemoveSkewnessKurtosis()),
                   ('StandardizeStandardScaler', Standardize(RobustScaler()))]
transform_pipeline = Pipeline(transform_steps)

X = transform_pipeline.transform(X)

/home/vivi/FDA/src/models/transformation.py:180: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[i] = self.scalar.fit_transform(X[i].values.reshape(-1,1))


In [7]:
len(data.columns)

82

In [5]:
# Grid Search CV
import numpy as np
import pandas as pd
import scipy as sp
import copy,os,sys,psutil
import lightgbm as lgb
from lightgbm.sklearn import LGBMRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import dump_svmlight_file
from sklearn.linear_model import LogisticRegression
 
from sklearn import metrics   #Additional scklearn functions
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from grid_search_cv import *

gsearch = RandomForest_Grid_CV(X,y, RandomForest_param)

KeyboardInterrupt: 

In [ ]:
d2=pd.DataFrame(X['Internalpatientid'])

In [ ]:
d2['new'] =[0]*84536

In [ ]:
data.columns

Index(['Internalpatientid', 'num_stays', 'stay_length', 'num_unique_units',
       'num_transfers', 'num_cvd_admission', 'CVD',
       'unique_admitting_specialty', 'unique_discharging_specialty',
       'DOMICILIARY', 'MEDICINE', 'NHCU', 'NON-COUNT', 'OTHERS', 'PSYCHIATRY',
       'SURGERY', 'Age 20-40', 'Age 40-60', 'Age 60-80', 'Age 80-100',
       'Age 100-120', 'age_mean', 'age_std', 'age_min', 'age_max', 'stay_min',
       'stay_max', 'stay_mean', 'stay_std', 'freq', 'Medical', 'Mental',
       'Others_Specialty', 'Rehab', 'Gerontology', 'CVD_readmission',
       'readmission within 300 days', 'total_procedure', 'num_surgery_pro',
       'Ethnicity', 'Gender', 'Races', 'Ethnicity_0', 'Ethnicity_1',
       'Ethnicity_2', 'Races_0', 'Races_1', 'Races_2', 'Races_3',
       'num_immunization', 'Num med per admission mean',
       'Num med per admission min', 'Num med per admission max',
       'Total medications', 'mean age at specailty', 'specialty count',
       'period mean', 'per

In [15]:
import pandas as pd
path = '/home/daisy/FDA_Dataset/final_cvd_mortality_train_1.csv'
data = pd.read_csv(path).iloc[:,1:]

In [14]:
data.columns

Index(['Internalpatientid', 'num_stays', 'stay_length', 'num_unique_units',
       'num_transfers', 'num_cvd_admission', 'CVD',
       'unique_admitting_specialty', 'unique_discharging_specialty',
       'DOMICILIARY', 'MEDICINE', 'NHCU', 'NON-COUNT', 'OTHERS', 'PSYCHIATRY',
       'SURGERY', 'Age 20-40', 'Age 40-60', 'Age 60-80', 'Age 80-100',
       'Age 100-120', 'age_mean', 'age_std', 'age_min', 'age_max', 'stay_min',
       'stay_max', 'stay_mean', 'stay_std', 'freq', 'Medical', 'Mental',
       'Others_Specialty', 'Rehab', 'Gerontology', 'CVD_readmission',
       'next_readmission_time', 'Discharge date',
       'readmission within 300 days', 'total_procedure', 'num_surgery_pro',
       'Ethnicity', 'Gender', 'Races', 'Ethnicity_0', 'Ethnicity_1',
       'Ethnicity_2', 'Races_0', 'Races_1', 'Races_2', 'Races_3',
       'num_immunization', 'Num med per admission mean',
       'Num med per admission min', 'Num med per admission max',
       'Total medications', 'mean age at specail

In [2]:
X = data.drop(columns = ['Internalpatientid', 'CVD_readmission', 'readmission within 300 days'])
y = column_or_1d(data[['readmission within 300 days']])
    
# Transform Data
transform_steps = [("ImputeNumeric", ImputeNumeric()),
        ('RemoveSkewnessKurtosis', RemoveSkewnessKurtosis()),
        ('StandardizeStandardScaler', Standardize(RobustScaler()))]
transform_pipeline = Pipeline(transform_steps)

X = transform_pipeline.transform(X)

KeyError: "['CVD_readmission', 'readmission within 300 days'] not found in axis"

In [4]:
import pickle
clf = pickle.load(open('/home/vivi/FDA/models/RandomForest_mortality.sav','rb'))
clf.feature_names_in_

array(['Age 60-80 hypotension_log_rob_scaled',
       'num_surgery_pro_log_rob_scaled', 'Age 00-20_log_rob_scaled',
       'specialty medical count_log_rob_scaled',
       'Age 40-60 hypertension_log_rob_scaled',
       'Age 100-120 healthy_log_rob_scaled', 'freq_log_rob_scaled',
       'Age 40-60 hypotension_log_rob_scaled',
       'Age 100-120 hypotension_log_rob_scaled',
       'lab_freq_log_rob_scaled', 'Age 100-120_log_rob_scaled',
       'Num med per admission max_log_rob_scaled',
       'period std_log_rob_scaled',
       'Age 20-40 hypertension_log_rob_scaled',
       'Age 60-80 hypertension_log_rob_scaled',
       'Num med per admission mean_log_rob_scaled',
       'Age 80-100 healthy_log_rob_scaled',
       'total_procedure_log_rob_scaled', 'period mean_log_rob_scaled',
       'Age 20-40 healthy_log_rob_scaled', 'lab_count_log_rob_scaled',
       'Age 20-40 hypotension_log_rob_scaled',
       'Num med per admission min_log_rob_scaled',
       'specialty support count_log_rob_

In [4]:
str = 'Age 20-40 hypertension_log_rob_scaled'
suffix = "_log_rob_scaled";
result=str.endswith(suffix)

In [5]:
result


True

In [20]:
import pandas as pd


from sklearn.utils.validation import column_or_1d
from transformation import *
path =  '/home/daisy/FDA_Dataset/inal_cvd_mortality_quality_1'
data = pd.read_csv(path).iloc[:,1:]
patientId = pd.DataFrame(data['Internalpatientid'])
X = data.drop(columns = ['Internalpatientid', 'CVD_readmission', 'readmission within 300 days'])
y = column_or_1d(data[['readmission within 300 days']])
transform_steps = [("ImputeNumeric", ImputeNumeric()),
            ('RemoveSkewnessKurtosis', RemoveSkewnessKurtosis(clf.feature_names_in_)),
            ('StandardizeStandardScaler', Standardize(RobustScaler()))]
transform_pipeline = Pipeline(transform_steps)

X = transform_pipeline.transform(X)
print(X.columns)
X = X[clf.feature_names_in_]


FileNotFoundError: [Errno 2] No such file or directory: '/home/daisy/FDA_Dataset/inal_cvd_mortality_quality_1'

In [21]:
data.columns


Index(['Internalpatientid', 'num_stays', 'stay_length', 'num_unique_units',
       'num_transfers', 'num_cvd_admission', 'CVD',
       'unique_admitting_specialty', 'unique_discharging_specialty',
       'DOMICILIARY', 'MEDICINE', 'NHCU', 'NON-COUNT', 'OTHERS', 'PSYCHIATRY',
       'SURGERY', 'Age 20-40', 'Age 40-60', 'Age 60-80', 'Age 80-100',
       'Age 100-120', 'age_mean', 'age_std', 'age_min', 'age_max', 'stay_min',
       'stay_max', 'stay_mean', 'stay_std', 'freq', 'Medical', 'Mental',
       'Others_Specialty', 'Rehab', 'Gerontology', 'CVD_readmission',
       'next_readmission_time', 'Discharge date',
       'readmission within 300 days', 'total_procedure', 'num_surgery_pro',
       'Ethnicity', 'Gender', 'Races', 'Ethnicity_0', 'Ethnicity_1',
       'Ethnicity_2', 'Races_0', 'Races_1', 'Races_2', 'Races_3',
       'num_immunization', 'Num med per admission mean',
       'Num med per admission min', 'Num med per admission max',
       'Total medications', 'mean age at specail

In [3]:
clf.predict(X)

array([0, 1, 1, ..., 0, 0, 0])

In [20]:
data['total_procedure']

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
         ..
22250   NaN
22251   NaN
22252   NaN
22253   NaN
22254   NaN
Name: total_procedure, Length: 22255, dtype: float64

In [6]:
import pickle
clf = pickle.load(open('/home/vivi/FDA/models/RandomForest_mortality.sav','rb'))
len(clf.feature_names_in_)

54

In [22]:
import pandas as pd


from sklearn.utils.validation import column_or_1d
from transformation import *
path =  '/home/daisy/FDA_Dataset/inpatient_CVD_final_test_1.csv'
data = pd.read_csv(path).iloc[:,1:]
patientId = pd.DataFrame(data['Internalpatientid'])
X = data.drop(columns = ['Internalpatientid', 'died_within_125days'])
y = column_or_1d(data[['died_within_125days']])
transform_steps = [("ImputeNumeric", ImputeNumeric()),
            ('RemoveSkewnessKurtosis', RemoveSkewnessKurtosis(clf.feature_names_in_)),
            ('StandardizeStandardScaler', Standardize(RobustScaler()))]
transform_pipeline = Pipeline(transform_steps)

X = transform_pipeline.transform(X)
print(X.columns)
X = X[clf.feature_names_in_]

KeyError: "['died_within_125days'] not found in axis"

In [24]:
data.columns

Index(['Internalpatientid', 'num_stays', 'stay_length', 'num_unique_units',
       'num_transfers', 'num_cvd_admission', 'CVD',
       'unique_admitting_specialty', 'unique_discharging_specialty',
       'DOMICILIARY', 'MEDICINE', 'NHCU', 'NON-COUNT', 'OTHERS', 'PSYCHIATRY',
       'SURGERY', 'Age 20-40', 'Age 40-60', 'Age 60-80', 'Age 80-100',
       'Age 100-120', 'age_mean', 'age_std', 'age_min', 'age_max', 'stay_min',
       'stay_max', 'stay_mean', 'stay_std', 'freq', 'Medical', 'Mental',
       'Others_Specialty', 'Rehab', 'Gerontology', 'CVD_readmission',
       'readmission within 300 days', 'total_procedure', 'num_surgery_pro',
       'Ethnicity', 'Gender', 'Races', 'Ethnicity_0', 'Ethnicity_1',
       'Ethnicity_2', 'Races_0', 'Races_1', 'Races_2', 'Races_3',
       'num_immunization', 'Num med per admission mean',
       'Num med per admission min', 'Num med per admission max',
       'Total medications', 'mean age at specailty', 'period mean',
       'period std', 'specialt

In [69]:
import pickle
clf = pickle.load(open('/home/vivi/FDA/models/LogisticRegression_mortality_cvd.sav','rb'))
len(clf.feature_names_in_)

78

In [82]:
import pandas as pd


from sklearn.utils.validation import column_or_1d
from transformation import *
path =  '/home/daisy/FDA_Dataset/final_cvd_mortality_train_1.csv'
data = pd.read_csv(path).iloc[:,1:]
patientId = pd.DataFrame(data['Internalpatientid'])
X = data.drop(columns = ['Internalpatientid','CVD_readmission','readmission within 300 days'])
transform_steps = [("ImputeNumeric", ImputeNumeric()),
            ('RemoveSkewnessKurtosis', RemoveSkewnessKurtosis(clf.feature_names_in_)),
            ('StandardizeStandardScaler', Standardize(RobustScaler()))]
transform_pipeline = Pipeline(transform_steps)

X = transform_pipeline.transform(X)
print(X.columns)
X = X[clf.feature_names_in_]

KeyError: "['CVD_readmission', 'readmission within 300 days'] not found in axis"

In [83]:
data.columns

Index(['Internalpatientid', 'num_stays', 'stay_length', 'num_unique_units',
       'num_transfers', 'num_cvd_admission', 'CVD',
       'unique_admitting_specialty', 'unique_discharging_specialty',
       'DOMICILIARY', 'MEDICINE', 'NHCU', 'NON-COUNT', 'OTHERS', 'PSYCHIATRY',
       'SURGERY', 'Age 20-40', 'Age 40-60', 'Age 60-80', 'Age 80-100',
       'Age 100-120', 'age_mean', 'age_std', 'age_min', 'age_max', 'stay_min',
       'stay_max', 'stay_mean', 'stay_std', 'freq', 'Medical', 'Mental',
       'Others_Specialty', 'Rehab', 'Gerontology', 'died_by_cvd',
       'Age at death', 'total_procedure', 'num_surgery_pro', 'Ethnicity',
       'Gender', 'Races', 'Ethnicity_0', 'Ethnicity_1', 'Ethnicity_2',
       'Races_0', 'Races_1', 'Races_2', 'Races_3', 'num_immunization',
       'Num med per admission mean', 'Num med per admission min',
       'Num med per admission max', 'Total medications',
       'mean age at specailty', 'specialty count', 'period mean', 'period std',
       'specialt

In [72]:
len(clf.feature_names_in_)

78

In [80]:
import pandas as pd


from sklearn.utils.validation import column_or_1d
from transformation import *
path =  '/home/daisy/FDA_Dataset/final_allcause_mortality_quality_1.csv'
data = pd.read_csv(path).iloc[:,1:]
patientId = pd.DataFrame(data['Internalpatientid'])
X2 = data.drop(columns = ['Internalpatientid','CVD_readmission','readmission within 300 days'])
transform_steps = [("ImputeNumeric", ImputeNumeric()),
            ('RemoveSkewnessKurtosis', RemoveSkewnessKurtosis(clf.feature_names_in_)),
            ('StandardizeStandardScaler', Standardize(RobustScaler()))]
transform_pipeline = Pipeline(transform_steps)

X2 = transform_pipeline.transform(X)
print(X.columns)
X2 = X2[clf.feature_names_in_]

KeyError: "['CVD_readmission', 'readmission within 300 days'] not found in axis"

In [81]:
data.columns

Index(['Internalpatientid', 'num_visits', 'CVD', 'Age 00-20', 'Age 20-40',
       'Age 40-60', 'Age 60-80', 'Age 80-100', 'Age 100-120', 'age_mean',
       'age_std', 'freq', 'total_procedure', 'num_surgery_pro', 'Ethnicity',
       'Gender', 'Races', 'Ethnicity_0', 'Ethnicity_1', 'Ethnicity_2',
       'Races_0', 'Races_1', 'Races_2', 'Races_3', 'num_immunization',
       'Num med per admission mean', 'Num med per admission min',
       'Num med per admission max', 'Total medications',
       'mean age at specailty', 'specialty count', 'period mean', 'period std',
       'specialty medical count', 'specialty support count',
       'Ruca category encoded', 'Age 20-40 hypotension',
       'Age 40-60 hypotension', 'Age 60-80 hypotension',
       'Age 80-100 hypotension', 'Age 100-120 hypotension',
       'Age 20-40 healthy', 'Age 40-60 healthy', 'Age 60-80 healthy',
       'Age 80-100 healthy', 'Age 100-120 healthy', 'Age 20-40 hypertension',
       'Age 40-60 hypertension', 'Age 60-80 hy

In [23]:
import pandas as pd


from sklearn.utils.validation import column_or_1d
from transformation import *
path =  '/home/vivi/FDA/reports/test_predict_result.csv'
data = pd.read_csv(path).iloc[:,1:]

In [26]:
len(data['mortality_label'])

61008

In [28]:
import pickle
clf = pickle.load(open('/home/vivi/FDA/models/LogisticRegression_mortality_cvd.sav','rb'))
len(clf.feature_names_in_)

77

In [27]:
import pandas as pd


from sklearn.utils.validation import column_or_1d
from transformation import *
path =  '/home/daisy/FDA_Dataset/final_cvd_mortality_train_1.csv'
data = pd.read_csv(path).iloc[:,1:]
patientId = pd.DataFrame(data['Internalpatientid'])
X = data.drop(columns = ['Internalpatientid','died_by_cvd','Age at death'])
y = column_or_1d(data[['died_by_cvd']])
transform_steps = [("ImputeNumeric", ImputeNumeric()),
            ('RemoveSkewnessKurtosis', RemoveSkewnessKurtosis(clf.feature_names_in_)),
            ('StandardizeStandardScaler', Standardize(RobustScaler()))]
transform_pipeline = Pipeline(transform_steps)

X = transform_pipeline.transform(X)

X = X[clf.feature_names_in_]

/home/vivi/FDA/src/models/transformation.py:180: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[i] = self.scalar.fit_transform(X[i].values.reshape(-1,1))


In [30]:
result = clf.predict(X)

In [31]:
unique, counts = np.unique(result, return_counts=True)

In [32]:
unique, counts

(array([0, 1]), array([39776, 21232]))

In [34]:
unique, counts = np.unique(y, return_counts=True)
unique, counts 

(array([0, 1]), array([57199,  3809]))

In [35]:
39776/57199

0.6953967726708509